# Mask Detector

## Importing modules

In [1]:
import tensorflow 
from tensorflow import keras 
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import numpy as np
import os

## Loading dataset

# Organizing data

In [5]:
# define path and catogories for classification
path_dir = "D:/github/Mask Detection/mask_detector-master/data"
categories = ["with_mask", "without_mask"]

## Load images 

In [6]:
print("Loading the images")
# make arrays for storing dataset of images and it's label of 
# with mask and wihtout mask
dataset = []
label = []
# join path for preprocessing of images
for category in categories:
    path = os.path.join(path_dir, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        #print (img_path)
        Img = keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
        Img = keras.preprocessing.image.img_to_array(Img)
        Img = keras.applications.mobilenet_v2.preprocess_input(Img)

        dataset.append(Img)
        label.append(category)

print("All images loaded successfully")

Loading the images
All images loaded successfully


## Convert categories into binary format

In [7]:
# Convert two categories into binary format like 0,1
label_binarizer = LabelBinarizer()
# Transform multi-class labels to binary labels
label = label_binarizer.fit_transform(label)
# Converts a class vector (integers) to binary class matrix
label = keras.utils.to_categorical(label)

## Create arrays of dataset and labels

In [8]:
dataset = np.array(dataset, dtype="float32")
label = np.array(label)

## Split dataset into training and testing

In [9]:
(trainX, testX, trainY, testY) = train_test_split(dataset, label,
	test_size=0.20, stratify=label, random_state=42)

## Apply augmentation on images

In [10]:
#use augmentation to increase variation in dataset by various operation like shifting and rotating images
data_augmentation = keras.preprocessing.image.ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.07,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.10, 
	horizontal_flip=True,
	fill_mode="nearest")

# Creating model

## Load pretrained mobilenetv2 model

In [11]:
# input_shape is shape of image(height, width, channeli.e. 3 here RGB)
# include_top is for fully connected layer
# imagenet is weights from pre-trained model
pre_trained_model = keras.applications.MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=keras.layers.Input(shape=(224, 224, 3)))

# we will freeze the base model to prevent them from updation while training
for layer in pre_trained_model.layers:
	layer.trainable = False


9412608/9406464 [==============================] - 3s 0us/step


## Sequential model

In [12]:
model = tensorflow.keras.Sequential(
    [keras.Input(shape=(224, 224, 3)),
     pre_trained_model,
     keras.layers.MaxPooling2D(pool_size=(2,2)),
     keras.layers.Dense(720, activation="relu"),
     keras.layers.Dense(32, activation="relu"),
     keras.layers.Dropout(0.3),
     keras.layers.Flatten(),
     keras.layers.Dense(units=2,activation="softmax")
     ] 
) 

# Build the model 

In [13]:
model.build([None, 224, 224, 3])
#print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 3, 1280)        0         
_________________________________________________________________
dense (Dense)                (None, 3, 3, 720)         922320    
_________________________________________________________________
dense_1 (Dense)              (None, 3, 3, 32)          23072     
_________________________________________________________________
dropout (Dropout)            (None, 3, 3, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 288)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 5

## Compiling the model

In [14]:
model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.Adam(),
              metrics=["accuracy"])

## Train the model

In [15]:
# train the head of the network
EPOCHS =30
batchSize = 50
FIT = model.fit(
	data_augmentation.flow(trainX, trainY, batch_size=batchSize),
	steps_per_epoch=len(trainX) // batchSize,
	validation_data=(testX, testY),
	validation_steps=len(testX) // batchSize,
	epochs= EPOCHS)


Epoch 1/30
68/68 [==============================] - 90s 1s/step - loss: 0.2649 - accuracy: 0.9300 - val_loss: 0.0670 - val_accuracy: 0.9789
Epoch 2/30
68/68 [==============================] - 86s 1s/step - loss: 0.0480 - accuracy: 0.9923 - val_loss: 0.0349 - val_accuracy: 0.9895
Epoch 3/30
68/68 [==============================] - 84s 1s/step - loss: 0.0345 - accuracy: 0.9906 - val_loss: 0.0433 - val_accuracy: 0.9860
Epoch 4/30
68/68 [==============================] - 90s 1s/step - loss: 0.0310 - accuracy: 0.9907 - val_loss: 0.0407 - val_accuracy: 0.9930
Epoch 5/30
68/68 [==============================] - 89s 1s/step - loss: 0.0364 - accuracy: 0.9900 - val_loss: 0.0386 - val_accuracy: 0.9906
Epoch 6/30
68/68 [==============================] - 91s 1s/step - loss: 0.0253 - accuracy: 0.9931 - val_loss: 0.1459 - val_accuracy: 0.9626
Epoch 7/30
68/68 [==============================] - 95s 1s/step - loss: 0.0434 - accuracy: 0.9911 - val_loss: 0.1057 - val_accuracy: 0.9708
Epoch 8/30
68/68 [==

## Prediction of model

In [16]:
PREDICT = model.predict(testX, batch_size=batchSize)

## Check test batch accuracy

In [17]:
# Make 2 lists to contain max values for predicted and actual test values
pred_class = []
ac_class = []
for i in range(len(PREDICT)):
    pr = PREDICT[i].argmax()
    pred_class.append(pr)
    ac = testY[i].argmax()
    ac_class.append(ac)

In [18]:
# Make 2 lists to contain the predicted and actual output results gained from 
# comparing max values in last step with alotted strings     
output = {0:"with_mask", 1:"wihout_mask"}
pred_op = []
ac_op = []
for i in range(len(pred_class)):
   pred_op.append(output[pred_class[i]])
   ac_op.append(output[ac_class[i]])

In [19]:
# Increment counter if the actual and predicted values match. Then calculate 
# % accuracy of the first batch.
correct_pred = 0
for i in range(batchSize):
    if pred_op[i] == ac_op[i]:
        correct_pred += 1

print("Accuracy of batch = ", (correct_pred/batchSize)*100, "%")

Accuracy of batch =  98.0 %


# Saving the model

In [20]:
model.save("mask_detector.model", save_format="h5")
print("The model is saved successfully!")

The model is saved successfully!
